In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
from xgcm import Grid
import numpy as np
import pandas as pd
from scipy import signal

import IPython.display
import cmocean as cm
import sys, os, glob
import warnings
warnings.simplefilter("ignore") 

from IPython.display import Image

from dask.distributed import Client

from matplotlib.patches import Rectangle

In [ ]:
# Load workers:
client = Client(n_workers=8)
client

## Testing

In [ ]:
base = '/g/data/e14/rmh561/croco/'
base_data = base + 'archive/'
base_analysis = base + '/analysis/'

os.chdir(base_analysis + 'PAC12_75_cpl')
sys.path.append(base_analysis + 'PAC12_75_cpl/Processing_Scripts')
sys.path.append(base_analysis + 'PAC12_75_cpl/Processing_Scripts/Tracer_balance_code_LMaillard_v2')

In [ ]:
# Load custom scripts/tools:
import pac12_tools as ptools
import R_tools_fort as toolsF

In [ ]:
# Reload (if needed):
import importlib
importlib.reload(ptools)

# Test Lisa's heat budget code:

In [ ]:
data_day = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_day.nc').rename({'time_counter':'time'})
data_mon = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_mon.nc').rename({'time_counter':'time'})
data_dayTIW = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_day_TIW.nc').rename({'time_counter':'time'})
data_grd = xr.open_dataset(base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp05/20150201_20150228/croco_out_grd.nc')

In [ ]:
L1, M1 = 1, 147
[Mm,Lm] = np.shape(data_dayTIW.temp.isel(time=0).isel(s_rho_TIW=0))
N_TIW = np.shape(data_dayTIW.s_rho_TIW)
N = np.shape(data_mon.s_rho)
u = data_dayTIW.u.isel(x_TIW_u=slice(1,-1)).isel(time=0)
v = data_dayTIW.v.isel(y_TIW_v=slice(1,-1)).isel(time=0)
h = data_grd.h.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))
zeta = data_day.zeta.isel(x_rho=slice(L1,L1+Lm),y_rho=slice(M1,M1+Mm)).isel(time=0)
hc = data_grd.hc
Cs_r = data_grd.Cs_r
Cs_w = data_grd.Cs_w
pm = data_grd.pm.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))
pn = data_grd.pn.isel(x=slice(L1,L1+Lm),y=slice(M1,M1+Mm))

In [ ]:
z_r, z_w = toolsF.zlevs(h,zeta,hc,Cs_r,Cs_w)
w = toolsF.get_omega(np.transpose(u.values, (2, 1, 0)),np.transpose(v.values, (2, 1, 0)),np.transpose(z_r[:,:,39:],(1,0,2)),np.transpose(z_w[:,:,39:],(1,0,2)),pm.T,pn.T)#,lm=1298,mm=198,n=36)

# Test high-pass filter processing scripts

In [ ]:
# Testing high-pass filter stuff:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'
file_out = 'test.nc'
filt_width = 12.

data = xr.open_dataset(file_in,chunks={'time_counter':1}).rename({'time_counter':'time'})
data = ptools.create_coords_CROCO(data)
DT = len(data.time)

In [ ]:
# Testing high-pass filter stuff:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_mon.nc'
file_out = 'test.nc'
filt_width = 12.

data = xr.open_dataset(file_in,chunks={'time_counter':1})

In [ ]:
# Testing high-pass filter stuff:
file_in_day = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'
filt_width = 12.

ds = ptools.calc_zhp_std_variables(file_in_day,file_in_mon,file_out,filt_width)

In [ ]:
# Testing Gaussian rather than box-car filter:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'
file_out = 'test.nc'
filt_width = 12. # Box-car filter width in degrees
gauss_width = 6. # Gaussian filter standard deviation in degrees
cutratio = 0.1   # Minimum amplitude of gaussian filter to keep.

data = xr.open_dataset(file_in,chunks={'time_counter':1}).rename({'time_counter':'time'})
data = ptools.create_coords_CROCO(data)

SST = data.temp_surf.isel(time=0)

In [ ]:
SST_boxcar = ptools.zlp_filt(SST,filt_width,typ='box')
SST_gauss = ptools.zlp_filt(SST,gauss_width,typ='gau',cut=0.1)

In [ ]:
var = SST
width = gauss_width
cut = cutratio
#def zlp_filt_gauss(var,width,cut):
    # Calculate zonal filtered version of a variable

dims = var.dims
inds = [index for index,item in enumerate(dims) if item.startswith('x')]
if (len(inds) != 1):
    raise RuntimeError("Error in zhp_filt: less than or greater than 1 zonal dimension found")
else:
    x = dims[inds[0]]

dx = (var[x][1]-var[x][0]).values

sigx = int(width/dx)
nn = int(3*2*sigx+1)

xx = np.arange(0.,nn)
dd = np.sqrt(((xx-int(nn/2))**2)/sigx**2)
ww = 1./(np.sqrt(np.pi)*sigx)*np.exp(-dd**2)
keepww = np.argwhere((ww.ravel() >= max(ww.ravel())*cutratio)); nnok=len(keepww)
ww = ww.ravel()[keepww.astype('int')]
ww = ww/ww.sum()

weight = xr.DataArray(ww.ravel(), dims=['window'])

SST_gauss = var.rolling({x:nnok},center=True).construct('window').dot(weight)
SST_gauss

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
SST.plot(vmin=22.,vmax=28.,cmap='RdBu_r')
plt.gca().set_ylim([-10.,15.])
plt.title('Raw')
plt.subplot(2,2,2)
SST_boxcar.plot(vmin=22.,vmax=28.,cmap='RdBu_r')
plt.gca().set_ylim([-10.,15.])
plt.title('Box-car 12-degree filter')
plt.plot([200.,220.,220.,220.+filt_width,220.+filt_width,260.],[0.,0.,5.,5.,0.,0.],'-k')
plt.subplot(2,2,3)
SST_gauss.plot(vmin=22.,vmax=28.,cmap='RdBu_r')
plt.gca().set_ylim([-10.,15.])
plt.title('Gaussian 6-degree filter')
x = 220.+xx*dx
x = x.ravel()[keepww.astype('int')]
plt.plot(x,ww*1000.,'-k')
plt.subplot(2,2,4)
(SST_boxcar-SST_gauss).plot(vmin=-0.1,vmax=0.1,cmap='RdBu_r')
plt.gca().set_ylim([-10.,15.])
plt.title('Difference between box-car and gaussian filter')

# Test V*SST heat flux:


In [ ]:
# Testing Gaussian rather than box-car filter:
file_in = base_data + 'PAC12_75_cpl/PAC12_75_cpl_exp13/20150101_20150131/croco_out_day.nc'

data = xr.open_dataset(file_in,chunks={'time_counter':1}).rename({'time_counter':'time'})
data = ptools.create_coords_CROCO(data)

SST = data.temp_surf.isel(time=0)
V = data.v_surf.isel(time=0)
U = data.u_surf.isel(time=0)

In [ ]:
SST_hp = SST - ptools.zlp_filt(SST,6.)
V_hp = V - ptools.zlp_filt(V,6.)
U_hp = U - ptools.zlp_filt(U,6.)

In [ ]:
grid = Grid(data,coords={"y":{"center":"y_rho","inner":"y_v"},"x":{"center":"x_rho","inner":"x_u"}},periodic=False)

In [ ]:
VSST = grid.interp(V_hp,'y').rename({'x_v':'x_rho'})*SST_hp
USST = grid.interp(U_hp,'x').rename({'y_u':'y_rho'})*SST_hp

In [ ]:
VSST.plot()